# August 17, 2022: organize MAX ROIs into the 7 networks defined in Schaefer parcellation

In [1]:
import os 
import sys
from os.path import join as pjoin
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle, random

from nltools import mask
from nilearn import image, masking

# plotting
import matplotlib.pyplot as plt
plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr #CITE ITS PAPER IN YOUR MANUSCRIPT

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'network_dynamics')
results_dir = f"{proj_dir}/results"
month_dir = f"{proj_dir}/nb/aug22"

# folders
sys.path.insert(0, proj_dir)
import helpers.dataset_utils as dataset_utils

/home/govindas/venvs/expln-ai3.9/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


## Schaefer and MAX parcellations

In [2]:
class ARGS(): pass
args = ARGS()

args.n_parcels = 1000
schaefer_main_path = f"{proj_dir}/data/schaefer_parcellations/n_parcels_{args.n_parcels}"

In [3]:
'''
network order information
of Schaefer ROIs
'''
nw_order = pd.read_csv(
    f"{schaefer_main_path}/Schaefer2018_1000Parcels_7Networks_order_info.txt",
    header=None,
).iloc[0::2]
nw_order = nw_order.reset_index(drop=True)
nw_order = nw_order[0].apply(lambda s: s.split('_')[2]).to_numpy()
nw_order

array(['Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis', 'Vis',
       'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot',
       'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot',
       'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot',
       'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot',
       'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot', 'SomMot',
       'SomMot', 'SomMot', 'SomMot'

In [4]:
'''
schaefer parcellation
'''
schaefer_parcel = image.load_img(
    f"{schaefer_main_path}/Schaefer2018_1000Parcels_7Networks_order_FSLMNI152_2mm.nii"
).get_fdata()

schaefer_parcel.shape

(91, 109, 91)

In [5]:
'''
MAX parcellation
'''
max_parcel = image.load_img(
    f"/home/govindas/parcellations/MAX_85_ROI_masks/MAX_ROIs_final_gm_85.nii.gz"
).get_fdata()

max_parcel.shape

(91, 109, 91)

## associate MAX rois to the 7 networks + subcortical network

In [6]:
'''
associate MAX rois to the 7 networks + subcortical network
'''
def get_most_freq_nw(nws):
    if len(nws) == 0: return 'Subcort'
    nws, cts = np.unique(nws, return_counts=True)
    return nws[np.argmax(cts)]

max_rois = np.unique(max_parcel)[1:]
max_order = np.empty(shape=(max_rois.shape[0]), dtype=object)
for idx, roi in enumerate(max_rois):
    roi_mask = (max_parcel == roi)
    schaefer_region = schaefer_parcel * roi_mask
    nws = nw_order[np.unique(schaefer_region)[1:].astype(int) - 1]
    print(
        roi, nws, get_most_freq_nw(nws)
    )
    max_order[idx] = get_most_freq_nw(nws) 

1.0 ['Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Default'] Limbic
2.0 ['Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Limbic' 'Default'] Limbic
3.0 ['Cont' 'Cont' 'Cont' 'Default'] Cont
4.0 ['Cont' 'Default' 'Default'] Default
5.0 ['Default' 'Default' 'Default' 'Default'] Default
6.0 ['Default' 'Default' 'Default'] Default
7.0 ['SalVentAttn' 'Cont' 'Cont'] Cont
8.0 ['SalVentAttn' 'SalVentAttn' 'Cont'] SalVentAttn
9.0 ['SomMot' 'DorsAttn' 'DorsAttn' 'DorsAttn'] DorsAttn
10.0 ['SomMot' 'DorsAttn' 'DorsAttn'] DorsAttn
11.0 ['Limbic' 'Cont' 'Default' 'Default' 'Default'] Default
12.0 ['Default' 'Default' 'Default' 'Default' 'Default' 'Default'] Default
13.0 ['SalVentAttn' 'Cont' 'Cont' 'Cont' 'Default' 'Default' 'Default'
 'Default'] Default
14.0 ['SalVentAttn' 'SalVentAttn' 'SalVentAttn' 'Default' 'Default' 'Default'
 'Default' 'Default' 'Default'] Default
15.0 ['SomMot' 'SomMot' 'SalVentAttn' 'SalVentAttn' 'SalVentAttn' 'SalVentAttn'
 'SalVentAttn' 'Cont' 'Con

In [7]:
'''
save in the MAX README file
'''
max_readme_file = f"/home/govindas/parcellations/MAX_85_ROI_masks/README_MAX_ROIs_final_gm_85.txt"
max_readme_df = pd.read_csv(
    max_readme_file,
    sep='\t',
)
max_readme_df['Schaefer_network'] = max_order
display(max_readme_df)

max_readme_df.to_csv(
    max_readme_file,
    sep='\t',
    index=False
)

,Hemi,ROI,Index,Voxels,File_Name,Schaefer_network
0,R,med. OFC,1,407,r-mOFC_2mm_MidlineCut_Amunts.nii.gz,Limbic
1,L,med. OFC,2,339,l-mOFC_2mm_MidlineCut_Amunts.nii.gz,Limbic
2,R,lat. OFC,3,56,r-lOFC_p.6_2mm_Amunts.nii.gz,Cont
3,L,lat. OFC,4,62,l-lOFC_p.6_2mm_Amunts.nii.gz,Default
4,M,vmPFC1,5,67,vmPFC.nii.gz,Default
...,...,...,...,...,...,...
80,L,PAG,81,24,lPAG_Ezra.nii.gz,Subcort
81,R,CrusI-shackman,82,109,r-cerebellum-crusI_UTvUS_Shackman_sphere.nii.gz,Subcort
82,L,CrusI-shackman,83,122,l-cerebellum-crusI_UTvUS_Shackman_sphere.nii.gz,Subcort
83,R,CrusI-krienen,84,105,r-cerebellum-crusI_KrienenBuckner_sphere.nii.gz,Subcort


In [8]:
# max_readme_df = max_readme_df.drop(['Unnamed: 0'], axis=1)
# max_readme_df.to_csv(
#     max_readme_file,
#     sep='\t',
#     index=False
# )

## ROI ordering: group according to networks

In [10]:
'''
network grouping: sort ROIs
'''
roi_ordering = np.argsort(max_order)
nw_names = np.unique(np.sort(max_order))
display(nw_names)

roi_ordering

array(['Cont', 'Default', 'DorsAttn', 'Limbic', 'SalVentAttn', 'Subcort',
       'Vis'], dtype=object)

array([35, 37,  2, 38,  6, 40, 41, 16, 36, 24, 23, 22, 21, 20, 17, 42, 13,
       12, 11, 10, 60,  5,  4,  3, 62,  9,  8, 57, 59,  0,  1, 26, 25, 15,
        7, 14, 29, 52, 51, 50, 49, 47, 46, 45, 48, 43, 18, 19, 39, 27, 44,
       69, 70, 82, 71, 81, 80, 72, 74, 73, 75, 76, 78, 68, 77, 79, 67, 30,
       65, 31, 32, 33, 34, 28, 83, 66, 53, 55, 56, 58, 64, 54, 84, 61, 63])